# Test of Significance of Assessor Type

# Step 1 : Load libraries

In [4]:
# import required libraries for dataframe and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

# We dont Probably need the Gridlines. Do we? If yes comment this line
sns.set(style="ticks")
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
flatui = sns.color_palette(flatui)

# https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9
import scipy.stats as ss
from collections import Counter
import math 
from scipy import stats

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute


# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

# Step 2: Read and transform data

In [5]:
# Reading the data on which analysis needs to be done
df_main = pd.read_excel("SEED2019_RawData_yoy.xlsx", sheet_name='consolidated_data')
df_main.head()

,raw_score,Acquiring,Approaching,Progressing,Achieved Standards,Sort_PerformanceCode,2016-2018with Site Validation,2017-2018 with Site Validation,Comparison 2016-2018,Comparison 2017-2018,...,performance_value,evaluator_id,round_num,Comparison 2016-2019,Comparison 2017-2019,2019_concat_Name,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,7,0,0,0,1,4,1.0,1.0,3.0,2.0,...,4,34501,Community Assessment,4,3,ElementaryABLAYAN ESDalaguete II,NaN,NaN,NaN,2019_concat_Name
1,5,0,0,1,0,3,1.0,1.0,3.0,2.0,...,3,34501,Community Assessment,4,3,ElementaryABLAYAN ESDalaguete II,NaN,NaN,NaN,NaN
2,5,0,0,1,0,3,1.0,1.0,3.0,2.0,...,3,34501,Community Assessment,4,3,ElementaryABLAYAN ESDalaguete II,NaN,NaN,NaN,NaN
3,7,0,0,0,1,4,1.0,1.0,3.0,2.0,...,4,34501,Community Assessment,4,3,ElementaryABLAYAN ESDalaguete II,NaN,NaN,NaN,NaN
4,7,0,0,0,1,4,1.0,1.0,3.0,2.0,...,4,34501,Community Assessment,4,3,ElementaryABLAYAN ESDalaguete II,NaN,NaN,NaN,NaN


In [6]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589365 entries, 0 to 589364
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   raw_score                       589365 non-null  int64  
 1   Acquiring                       589365 non-null  int64  
 2   Approaching                     589365 non-null  int64  
 3   Progressing                     589365 non-null  int64  
 4   Achieved Standards              589365 non-null  int64  
 5   Sort_PerformanceCode            589365 non-null  int64  
 6   2016-2018with Site Validation   85598 non-null   float64
 7   2017-2018 with Site Validation  85598 non-null   float64
 8   Comparison 2016-2018            362088 non-null  float64
 9   Comparison 2017-2018            362088 non-null  float64
 10  allSchools                      589365 non-null  object 
 11  allschools_year                 589365 non-null  object 
 12  2018_concatSchoo

Check if there is a significant difference between the type of assessor: https://towardsdatascience.com/chi-square-test-for-independence-in-python-with-examples-from-the-ibm-hr-analytics-dataset-97b9ec9bb80a

In [7]:
df_chi = df_main[df_main['Year'] == 2019]
df_chi = df_chi[['school_name', 'round_num', 'indicator', 'raw_score']]
df_chi. rename(columns = {'round_num':'assessor'}, inplace = True)
df_chi = df_chi.groupby(['school_name', 'assessor', 'indicator']).mean('raw_score').reset_index()
df_chi['raw_score'] = np.where(df_chi['raw_score'] >= 7, "A", 
                            np.where(df_chi['raw_score'] >= 5, "B", 
                            np.where(df_chi['raw_score'] >= 3, "C", 
                            np.where(df_chi['raw_score'] >= 1, "D", "None"))))
#df_chi = df_chi.groupby(['school_name', 'Year', 'indicator']).mean('raw_score').reset_index()
#df_chi = df_chi.pivot(index='school_name', columns='indicator', values='raw_score').reset_index()

In [8]:
df_chi['raw_score'].value_counts()

B    32755
C    21371
A    15233
D     4857
Name: raw_score, dtype: int64

In [9]:
df_test = df_chi[df_chi['indicator'] == 'Drop-out Rate']

In [10]:
df_test

,school_name,assessor,indicator,raw_score
13,ABLAYAN ES,Community Assessment,Drop-out Rate,C
61,ABLAYAN ES,Site Validation,Drop-out Rate,C
109,ABUCAYAN ES,Community Assessment,Drop-out Rate,B
157,ABUCAYAN ES,Site Validation,Drop-out Rate,D
205,ABUGON ES,Community Assessment,Drop-out Rate,B
...,...,...,...,...
73989,caduawan ES,Site Validation,Drop-out Rate,B
74037,can-aga es,Community Assessment,Drop-out Rate,C
74085,catag PS,Community Assessment,Drop-out Rate,A
74133,catarman elementary school,Community Assessment,Drop-out Rate,D


In [11]:
ct = pd.crosstab(df_test.assessor, df_test.raw_score, margins=True)

In [12]:
row_sum = ct.iloc[0:2,4].values
exp = []
for j in range(2):
    for val in ct.iloc[2,0:4].values:
        exp.append(val * row_sum[j] / ct.loc['All', 'All'])
print(exp)

[287.7425614489004, 506.0058214747736, 226.6849935316947, 64.56662354463131, 122.25743855109961, 214.9941785252264, 96.3150064683053, 27.433376455368695]


In [13]:
obs = np.append(ct.iloc[0][0:4].values, ct.iloc[1][0:4].values)
obs

array([267, 532, 231,  55, 143, 189,  92,  37], dtype=int64)

In [14]:
row_sum = ct.iloc[0:2,4].values
exp = []
for j in range(2):
    for val in ct.iloc[2,0:4].values:
        exp.append(val * row_sum[j] / ct.loc['All', 'All'])
exp

[287.7425614489004,
 506.0058214747736,
 226.6849935316947,
 64.56662354463131,
 122.25743855109961,
 214.9941785252264,
 96.3150064683053,
 27.433376455368695]

In [15]:
chi_sq_stats = ((obs - exp)**2/exp).sum()
chi_sq_stats

14.52173801348917

In [16]:
dof = (len(row_sum)-1)*(len(ct.iloc[2,0:4].values)-1)
dof

3

In [17]:
1 - stats.chi2.cdf(chi_sq_stats, dof)

0.002274515958943235

In [18]:
obs = np.array([ct.iloc[0][0:4].values,
                  ct.iloc[1][0:4].values])
stats.chi2_contingency(obs)[0:3]

(14.52173801348917, 0.0022745159589431887, 3)